Import required libraries

In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

Read data file 

In [2]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
target_col = 'Strength'
predictors = concrete_data.drop([target_col],axis=1).values
target = concrete_data[target_col].values
n_cols=predictors.shape[1]
cycles = 50
num_epochs = 50

A. Build a baseline model (5 marks) 
Use the Keras library to build a neural network with the following:
- One hidden layer of 10 nodes, and a ReLU activation function
- Use the adam optimizer and the mean squared error  as the loss function.
1. Randomly split the data into a training and test sets by holding 30% of the data for testing. You can use the train_test_splithelper function from Scikit-learn.
2. Train the model on the training data using 50 epochs.
3. Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength. You can use the mean_squared_error function from Scikit-learn.
4. Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.
5. Report the mean and the standard deviation of the mean squared errors.



In [3]:
def create_model(predictors, target, num_hidden = 1, num_epochs = 50):

    # create regression model
    model = Sequential()
    for i in range(num_hidden):
      model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
        
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')

    res = []
    for i in range(cycles):
        #Randomly split the data into a training set (70%) and a test set (30%):  
        X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3)
        #Train and test the model at the same time
        model.fit(X_train, y_train, epochs=num_epochs, verbose=0)
        #Add mean_squared_error for every cycle 
        res.append(model.evaluate(X_test, y_test, verbose=0))
    return np.mean(res), np.std(res)  

In [4]:
mean_before_norm, stdd_before_norm = create_model(predictors, target)
print("Mean      of least square errors for {} cycles is  {}".format(cycles, mean_before_norm))
print("Std. Dev. of least square errors for {} cycles is  {}".format(cycles, stdd_before_norm))

Mean      of least square errors for 50 cycles is  112.8608405303955
Std. Dev. of least square errors for 50 cycles is  181.47491148967964


B. Normalize the data (5 marks) 

Repeat Part A but use a normalized version of the data. Recall that one way to normalize the data is by subtracting the mean from the individual predictors and dividing by the standard deviation.

How does the mean of the mean squared errors compare to that from Step A?

In [5]:
predictors_norm = (predictors - predictors.mean())/predictors.std()
mean_with_norm, stdd_with_norm = create_model(predictors_norm, target)  
print("Mean      of least square errors for {} cycles after normalisation is  {}".format(cycles, mean_with_norm))
print("Std. Dev. of least square errors for {} cycles after normalisation is  {}".format(cycles, stdd_with_norm))


Mean      of least square errors for 50 cycles after normalisation is  105.66792976379395
Std. Dev. of least square errors for 50 cycles after normalisation is  48.72150774300432


C. Increate the number of epochs (5 marks)

Repeat Part B but use 100 epochs this time for training.

How does the mean of the mean squared errors compare to that from Step B?

In [6]:
mean_with_norm100, stdd_with_norm100 = create_model(predictors_norm, target, num_epochs=100)  
print("Mean      of least square errors for {} cycles after normalisation with 100 epochs is  {}".format(cycles, mean_with_norm100))
print("Std. Dev. of least square errors for {} cycles after normalisation with 100 epochs is  {}".format(cycles, stdd_with_norm100))
print('Performance comparison for different number of epochs :\n')
df100 = pd.DataFrame([['mean', mean_with_norm, mean_with_norm100], ['std. deviation', stdd_with_norm, stdd_with_norm100]], columns=['Measure', "50 epochs", "100 epochs"], index=None)
df100.set_index(['Measure'], inplace=True)
print(df100)



Mean      of least square errors for 50 cycles after normalisation with 100 epochs is  67.23333320617675
Std. Dev. of least square errors for 50 cycles after normalisation with 100 epochs is  41.89148101371774
Performance comparison for normalised predictors :

                 50 epochs  100 epochs
Measure                               
mean            105.667930   67.233333
std. deviation   48.721508   41.891481


D. Increase the number of hidden layers (5 marks)

Repeat part B but use a neural network with the following instead:

- Three hidden layers, each of 10 nodes and ReLU activation function.

How does the mean of the mean squared errors compare to that from Step B?

In [7]:
mean_with_norm3, stdd_with_norm3 = create_model(predictors_norm, target, num_hidden = 3) 
print("Mean      of least square errors for {} cycles after normalisation with 3 hidden layers is  {}".format(cycles, mean_with_norm3))
print("Std. Dev. of least square errors for {} cycles after normalisation with 3 hidden layers is  {}".format(cycles, stdd_with_norm3))
print('Performance comparison for different hidden layers :\n')
df3 = pd.DataFrame([['mean', mean_with_norm, mean_with_norm3], ['std. deviation', stdd_with_norm, stdd_with_norm3]], columns=['Measure', "1 hidden layer", "3 hidden layers"], index=None)
df3.set_index(['Measure'], inplace=True)
print(df3) 


Mean      of least square errors for 50 cycles after normalisation with 3 hidden layers is  61.316516418457034
Std. Dev. of least square errors for 50 cycles after normalisation with 3 hidden layers is  33.798477860484745
Performance comparison for normalised predictors :

                1 hidden layer  3 hidden layers
Measure                                        
mean                105.667930        61.316516
std. deviation       48.721508        33.798478
